<a href="https://colab.research.google.com/github/NK-Mikey/Data_Analysis/blob/main/Project_AR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project: Automate Report Process**

This project automates the end-to-end creation and delivery of a financial analytics report using Python. The system pulls real-time market data, processes it through a reproducible data pipeline, generates professional visualizations and risk metrics, and automatically emails a PDF report on a scheduled basis.

## 1. GDrive Connection

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


Connecting google drive for storing the pdf report output from this notebook.



## 2. Configuration

In [13]:
# Configure Tickers & Weights
TICKERS = ["AAPL", "MSFT", "SPY"] # Portfolio + Benchmark
WEIGHTS = {"AAPL": 0.4, "MSFT": 0.4, "SPY": 0.2} # Ensuring all the weights sum to 1
LOOKBACK_DAYS = 365 * 3 # Length of data retrival
REPORT_DIR = "/content/gdrive/MyDrive/Portfolio_reports"

1. We are choosing Apple, Microsoft, and SPY as the portfolio stocks.

    > SPY is an exchange-traded fund (ETF) that tracks the S&P 500 index, which represents 500 of the largest publicly traded companies in the U.S.

2. We assign 40% to Apple, 40% to Microsoft, and 20% to SPY so the script can calculate how much each stock contributes to our portfolio’s total performance.

3. We limit how many days of stock prices we download. In this case, we only retrieve the last 3 years of data.

4. We save the final PDF report from this project into Google Drive.

In [38]:
# Email Configuration (Colab Testing Only)
from getpass import getpass
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
EMAIL_USER = "naveenxkaran@gmail.com"
EMAIL_PASS = getpass("Email App Password: ")
RECEIVER_EMAIL = "naveenkaran26@gmail.com"

Email App Password: ··········


1. Simple Mail Transfer Protocol (SMTP) is the standard protocol used to send emails either from a client to an email server or from one server to another.

    > Under the hood, Transmission Control Protocol (TCP) and Transport Layer Security (TLS) work together: TCP ensures the email delivered reliably and in the correct order while TLS encrypts the connection so the data remains secure and protected from eavesdropping.

2. A port is a virtual “door” on a server that handles specific types of network traffic. Each port is assigned to a particular protocol or service, ensuring data reaches the correct destination.

## 3. Data Ingestion

In [4]:
# Import Libraries
import yfinance as yf
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

# Create a directory for reports
os.makedirs(REPORT_DIR, exist_ok=True)

# Define the date range
end = datetime.now()
start = end - timedelta(days = LOOKBACK_DAYS)

# Define a function to fetch stock prices
def fetch_prices(tickers, start, end):
  data = {}
  for t in tickers:
    df = yf.download(t, start = start.strftime("%Y-%m-%d"),
                     end = end.strftime("%Y-%m-%d"),
                     progress = False, auto_adjust = True)
    if df.empty:
      print(f"Warning: no data for {t}")
    else:
        data[t] = df
  return data

# Fetch prices using the function
prices = fetch_prices(TICKERS, start, end)

# Quick sanity check
len(prices), list(prices.keys())

(3, ['AAPL', 'MSFT', 'SPY'])

1. The financial stock price data is retrieved from Yahoo Finance using the Python library `yfinance`.

2. The `os` module is used to create a folder for storing outputs (such as reports and charts), only if the folder does not already exist.

3. A date range is created by calculating the start date and end date using `datetime` and `timedelta`.

   > `datetime` provides the current date and time, while `timedelta` is used to perform date arithmetic, such as calculating the difference between two dates.

4. A function is defined to fetch historical stock price data from Yahoo Finance for the predefined tickers within the specified date range. The function also checks whether any ticker returns empty data. If valid data is retrieved, it is stored in a dictionary named `data`.

   > `auto_adjust=True` ensures that the fetched price data is automatically adjusted for stock splits and dividends, which is suitable for accurately calculating financial KPIs over time.

5. The function is then called, and a quick validation is performed by checking the number of tickers retrieved and listing their keys to ensure they match the predefined tickers.



## 4. Validation & Processing

In [5]:
# Define a function to extract only Close price for all tickers
def validate_and_align(prices_dict):

    # Create a dictionary of Close price series for each ticker
    close_dict = {}
    for ticker, df in prices_dict.items():

        # Extract Close column based on column type
        if isinstance(df.columns, pd.MultiIndex):
            # MultiIndex columns
            if "Close" in df.columns.get_level_values(0):
                close_dict[ticker] = df.xs("Close", level=0, axis=1)[ticker]
            else:
                print(f"{ticker} has no 'Close' in MultiIndex columns")
        else:
            # Single-level columns
            if "Close" in df.columns:
                close_dict[ticker] = df["Close"]
            else:
                print(f"{ticker} has no 'Close' column")

    # Convert dictionary to a single DataFrame
    close_df = pd.DataFrame(close_dict)

    # Drop rows where all values are NaN/empty
    close_df = close_df.dropna(how="all")

    # Filling small gaps using forward and backward fills
    close_df = close_df.ffill().bfill()

    return close_df

# Calling the function to extract only the Close prices for all tickers
close_prices = validate_and_align(prices)

# Display the dataframe
close_prices.tail()

,AAPL,MSFT,SPY
Date,,,
2026-01-20,246.699997,454.519989,677.580017
2026-01-21,247.649994,444.109985,685.400024
2026-01-22,248.350006,451.140015,688.979980
2026-01-23,248.039993,465.950012,689.229980
2026-01-26,255.410004,470.279999,692.729980


1. A function is defined to extract the “Close” price for each ticker from the `prices` dictionary and combine them into a single DataFrame.
    > The function is designed to handle both MultiIndex and single-level columns returned by `yfinance`. When the data uses a MultiIndex, the `.xs()` (cross-section) method is used to extract the `"Close"` price from the appropriate index level.
2. Rows where all ticker values are missing are removed using `dropna(how="all")`.
3. Fill small gaps here and there using forward and backward fills.
    > Any small gaps in the data are handled using forward fill (`ffill`) and backward fill (`bfill`). Forward fill propagates the last available value forward, while backward fill fills missing values at the beginning by using the next available value.
4. Finally, the last five observations are displayed to validate the output.

## 5. Computing Returns, Portfolio Returns and Metrics

In [6]:
# Convert close prices into daily returns
rets = close_prices.pct_change().dropna()

# Create portfolio returns using weights
weights_vector = np.array([WEIGHTS.get(t, 0.0) for t in rets.columns])
port_rets = rets.dot(weights_vector)

# Define trading days
TRADING_DAYS = 252 # U.S. markets trade about 252 days per year

# Create a function to compute Annualized Return
def annualized_return(series):
  cumulative = (1 + series).prod()
  n = len(series) / TRADING_DAYS
  return cumulative ** (1/n) - 1

# Create a function to compute Annualized volatility (risk)
def annualized_vol(series):
  return series.std() * np.sqrt(TRADING_DAYS)

# Create a function to compute Sharpe Ratio (risk-adjusted return)
def sharpe_ratio(series, risk_free = 0.0):
  ar = annualized_return(series)
  avol = annualized_vol(series)
  if avol == 0:
    return np.nan
  return (ar - risk_free) / avol

# Create a function to compute Maximum Drawdown (worst loss)
def max_drawdown(series):
  cum = (1 + series).cumprod()
  running_max = cum.cummax()
  drawdown = (cum - running_max)/running_max
  return drawdown.min()

# Create a function to compute Value at Risk (VaR)
def var_historic(series, level = 0.95):
  return -np.percentile(series.dropna(), (1 - level)* 100)

# Create a function to compute Sortino Ratio (downside risk only)
def sortino_ratio(series, risk_free = 0.0):
  neg_rets = series[series < 0]
  downside_std = neg_rets.std() * np.sqrt(TRADING_DAYS)
  ar = annualized_return(series)
  if downside_std == 0:
    return np.nan
  return (ar - risk_free) / downside_std

# Computing portfolio metrics
metrics = {
    "Annual Return": annualized_return(port_rets),
    "Annual Volatility": annualized_vol(port_rets),
    "Sharpe Ratio": sharpe_ratio(port_rets),
    "Sortino Ratio": sortino_ratio(port_rets),
    "Maximum Drawdown": max_drawdown(port_rets),
    "Value at Risk (95%)": var_historic(port_rets, level = 0.95),
    "Value at Risk (99%)": var_historic(port_rets, level = 0.99),
}

# Computing per-asset metrics for comparison between tickers
asset_metrics = {}
for t in rets.columns:
  s = rets[t]
  asset_metrics[t] = {
      "annual_return": annualized_return(s),
      "annual_vol": annualized_vol(s),
      "sharpe": sharpe_ratio(s),
      "max_drawdown": max_drawdown(s),
  }

1. We compute the daily returns from the closing prices of each ticker to measure day-to-day percentage changes.

2. To calculate the portfolio return, which represents a single combined return for all tickers, we convert the predefined weight dictionary into a vector.

    >This allows us to compute a weighted average of individual asset returns using a dot product between the return matrix and the weight vector.

3. We define one trading year as 252 trading days, which reflects the typical number of trading days in U.S. financial markets and is used for annualizing returns and risk metrics.

4. We then create several functions to compute key investment performance and risk metrics, including:

   * Annualized Return
   * Annualized Volatility
   * Sharpe Ratio
   * Sortino Ratio
   * Maximum Drawdown
   * Value at Risk (VaR) at the 95% and 99% confidence levels

5. Finally we create two dictionaries which computes portfolio metrics and asset metrics.

## 6. Visualization

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

var_95 = var_historic(port_rets, level=0.95)
var_99 = var_historic(port_rets, level=0.99)

# 1. Price Chart
def save_price_chart(close_prices, out_path):
  plt.figure(figsize=(11,6))
  for col in close_prices.columns:
      plt.plot(close_prices.index, close_prices[col], label=col, linewidth=2)
  plt.title("Asset Performance Comparison", fontsize=25, pad=20)
  plt.ylabel("Indexed Price")
  plt.xlabel("Date")
  ax = plt.gca()
  for spine in ax.spines.values():
      spine.set_visible(False)
  ax.tick_params(left=False, bottom=False)
  plt.grid(True, axis='y', linestyle='--', alpha=0.4)
  plt.legend(frameon=False, loc='upper left', bbox_to_anchor=(1, 1))
  plt.tight_layout()
  plt.savefig(out_path)
  plt.close()

# 2. Portfolio Cumulative Returns
def save_cum_returns_chart(port_rets, out_path):
  cum = (1 + port_rets).cumprod() - 1
  plt.figure(figsize=(11,6))
  plt.plot(cum.index, cum * 100, color='#008080')
  plt.title("Portfolio Cumulative Return Over Time", fontsize=25, pad=20)
  plt.ylabel("Cumulative Return (%)")
  plt.xlabel("Date")
  ax = plt.gca()
  for spine in ax.spines.values():
      spine.set_visible(False)
  ax.tick_params(left=False, bottom=False)
  plt.grid(True, axis='y', linestyle='--', alpha=0.4)
  final_return = cum.iloc[-1] * 100
  plt.annotate(
      f"Final Return: {final_return:.1f}%",
      xy=(cum.index[-1], final_return),
      xytext=(-20, 30),
      textcoords="offset points",
      arrowprops=dict(arrowstyle="->", alpha=0.5),
      fontsize=10
  )
  plt.tight_layout()
  plt.savefig(out_path)
  plt.close()

# 3. Portfolio Drawdown
def save_drawdown_chart(port_rets, out_path):
  cum = (1 + port_rets).cumprod()
  running_max = cum.cummax()
  drawdown = (cum - running_max) / running_max
  max_dd = drawdown.min()
  max_dd_date = drawdown.idxmin()

  plt.figure(figsize=(11,4))
  plt.fill_between(drawdown.index, drawdown * 100, 0, alpha=0.15, color="#D0312D")
  plt.plot(drawdown.index, drawdown * 100, linewidth=1.5, color="#D0312D")
  plt.annotate(
      f"Max Drawdown: {max_dd:.2%}",
      xy=(max_dd_date, max_dd * 100),
      xytext=(10, 20),
      textcoords="offset points",
      arrowprops=dict(arrowstyle="->"),
      fontsize=9,
  )
  plt.title("Portfolio Drawdown Over Time", fontsize=25, pad=20)
  plt.ylabel("Drawdown (%)")
  plt.xlabel("Date")
  ax = plt.gca()
  for spine in ax.spines.values():
      spine.set_visible(False)
  ax.tick_params(left=False, bottom=False)
  plt.grid(True, axis='y', linestyle='--', alpha=0.4)
  plt.tight_layout()
  plt.savefig(out_path)
  plt.close()

# 4. Returns Distribution + VaR
def save_returns_distribution(port_rets, var_95, var_99, out_path):
  mean_ret = port_rets.mean()

  plt.figure(figsize=(9,4))
  plt.hist(port_rets, bins=50, density=True, alpha=0.65, color='#008080')
  plt.axvline(var_95, linestyle="--", linewidth=2, label=f"VaR 95% ({var_95:.2%})", color='#808080')
  plt.axvline(var_99, linestyle=":", linewidth=2, label=f"VaR 99% ({var_99:.2%})", color='#808080')
  plt.axvline(mean_ret, linestyle="-", linewidth=1.5, label=f"Mean ({mean_ret:.2%})", color='#808080')
  plt.title("Distribution of Daily Portfolio Returns", fontsize=20, pad=20)
  plt.xlabel("Daily Return")
  plt.ylabel("Probability Density")
  ax = plt.gca()
  for spine in ax.spines.values():
      spine.set_visible(False)
  ax.tick_params(left=False, bottom=False)
  plt.grid(True, axis='y', linestyle='--', alpha=0.4)
  plt.legend(frameon=False)
  plt.tight_layout()
  plt.savefig(out_path)
  plt.close()

# 5. Rolling Volatility
def save_rolling_volatility(port_rets, out_path, window=30):
  rolling_vol = port_rets.rolling(window).std() * np.sqrt(252)

  plt.figure(figsize=(11,4))
  plt.plot(rolling_vol.index, rolling_vol * 100, linewidth=2, label="Rolling Volatility", color='#008080')
  plt.fill_between(rolling_vol.index, rolling_vol * 100, alpha=0.15, color='#008080')
  plt.title("Portfolio Risk Over Time", fontsize=25, pad=20)
  plt.ylabel("Volatility (%)")
  plt.xlabel("Date")
  ax = plt.gca()
  for spine in ax.spines.values():
      spine.set_visible(False)
  ax.tick_params(left=False, bottom=False)
  ax.yaxis.set_major_formatter(mtick.PercentFormatter())
  plt.grid(True, axis='y', linestyle='--', alpha=0.4)
  plt.tight_layout()
  plt.savefig(out_path)
  plt.close()

# 6. Correlation Heatmap
def save_correlation_heatmap(rets, out_path):
  corr = rets.corr()

  plt.figure(figsize=(6,5))
  sns.heatmap(
    corr,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    center=0,
    linewidths=0.5,
    cbar_kws={"label": "Correlation"}
  )
  plt.title("Asset Return Correlation Matrix", fontsize=14, pad=20)
  plt.tight_layout()
  plt.savefig(out_path)
  plt.close()


In [8]:
# Define chart paths
chart_price_path = os.path.join(REPORT_DIR, "chart_price.png")
chart_cum_path = os.path.join(REPORT_DIR, "chart_cumulative_returns.png")
chart_dd_path = os.path.join(REPORT_DIR, "chart_drawdown.png")
chart_dist_path = os.path.join(REPORT_DIR, "chart_return_distribution.png")
chart_vol_path = os.path.join(REPORT_DIR, "chart_rolling_volatility.png")
chart_corr_path = os.path.join(REPORT_DIR, "chart_correlation.png")

# Execute the chart functions
save_price_chart(close_prices, chart_price_path)
save_cum_returns_chart(port_rets, chart_cum_path)
save_drawdown_chart(port_rets, chart_dd_path)
save_returns_distribution(port_rets, var_95, var_99, chart_dist_path)
save_rolling_volatility(port_rets, chart_vol_path)
save_correlation_heatmap(rets, chart_corr_path)

In this step, we created a comprehensive set of visualizations to analyze both portfolio performance and risk characteristics, including:

* Asset price performance over time
* Portfolio cumulative returns
* Maximum drawdown and drawdown trends
* Distribution of daily returns with risk measures
* Rolling volatility to capture changing risk levels
* Correlation between assets to assess diversification
* A KPI dashboard summarizing portfolio-level and asset-level metrics

All charts are programmatically generated and saved to Google Drive, enabling easy report compilation, sharing, and automation.



## 7. Build PDF Report

In [9]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.3 MB/s eta 0:00:00


In [10]:
# Import necessary libraries
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import Paragraph, SimpleDocTemplate, Spacer, Image, Table, TableStyle
from PIL import Image as PILImage
from datetime import datetime
import os

# Define function to scale charts while keeping aspect ratio
def scaled_image(path, max_width=7.5*inch, max_height=7*inch):
  img = PILImage.open(path)
  w, h = img.size
  aspect = h / w

  # Scale to fit max_width or max_height
  if w > h:
    width = max_width
    height = width * aspect
  else:
    height = max_height
    width = height / aspect

  return Image(path, width=width, height=height)

# Define function to create a PDF report
def create_pdf_report(report_path, metrics, asset_metrics, charts, config=None):
  doc = SimpleDocTemplate(
      report_path,
      pagesize=letter,
      rightMargin=36,
      leftMargin=36,
      topMargin=36,
      bottomMargin=36
  )
  styles = getSampleStyleSheet()
  story = []

  # Title
  story.append(Paragraph("Automated Portfolio Performance & Risk Report", styles['Title']))
  story.append(Spacer(1, 14))
  story.append(Paragraph(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} (GMT)", styles['Normal']))
  story.append(Spacer(1, 30))

  # Portfolio metrics table
  data = [["Metric", "Value"]]
  def fmt(x):
    return f"{x:.4f}" if isinstance(x, float) else str(x)
  for k, v in metrics.items():
      data.append([k, fmt(v)])
  tbl = Table(data, colWidths=[200, 200])
  tbl.setStyle(TableStyle([
      ('BACKGROUND', (0,0), (-1,0), colors.lightgrey),
      ('GRID', (0,0), (-1,-1), 0.5, colors.grey),
  ]))
  story.append(tbl)
  story.append(Spacer(1, 30))

  # Asset metrics table
  data2 = [["Asset", "Annualized Return", "Annualized Vol", "Sharpe", "Max Drawdown"]]
  for asset, m in asset_metrics.items():
      data2.append([
          asset,
          f"{m['annual_return']:.4f}",
          f"{m['annual_vol']:.4f}",
          f"{m['sharpe']:.4f}",
          f"{m['max_drawdown']:.4f}"
      ])
  tbl2 = Table(data2, colWidths=[120, 100, 100, 100, 100])
  tbl2.setStyle(TableStyle([
      ('BACKGROUND', (0,0), (-1,0), colors.lightgrey),
      ('GRID', (0,0), (-1,-1), 0.5, colors.grey),
  ]))
  story.append(tbl2)
  story.append(Spacer(1, 30))

  # Charts
  for chart_path in charts:
      if os.path.exists(chart_path):
          story.append(scaled_image(chart_path))
          story.append(Spacer(1, 65))

  # Build PDF
  doc.build(story)

# Generate report path
report_filename = f"portfolio_report_{datetime.now().strftime('%Y%m%d')}.pdf"
report_path = os.path.join(REPORT_DIR, report_filename)

# Call the function
create_pdf_report(
    report_path,
    metrics,
    asset_metrics,
    [
        chart_price_path,
        chart_cum_path,
        chart_dd_path,
        chart_dist_path,
        chart_vol_path,
        chart_corr_path
    ]
)

# Diplay path confirmation
report_path

'/content/gdrive/MyDrive/Portfolio_reports/portfolio_report_20260127.pdf'

In this step, we take all the metrics and visualizations we've created and package them into a professional, shareable PDF report. This ensures stakeholders or team members can view portfolio insights without accessing the raw code or data.

Key Features of this Step:
-  Dynamic Title & Timestamp: Automatically adds report title and current date/time for versioning.
-  Portfolio-Level Metrics Table:Summarizes key portfolio KPIs like annual return, volatility, Sharpe ratio, max drawdown, etc.
-  Asset-Level Metrics Table: Compares each asset in the portfolio with annualized return, volatility, Sharpe ratio, and drawdown.
-  Charts Integration: Embeds all relevant visualizations (price trends, cumulative returns, drawdowns, volatility, return distribution, correlation heatmap) into the PDF.
-  Automatic Chart Scaling: Maintains aspect ratio and scales charts to fit the page for readability.
-  Professional Layout: Uses spacing, fonts, and table styling for a clean and executive-ready appearance.

This step transforms raw analytics and visuals into a single, shareable report that can be distributed automatically via email or stored for record-keeping.




## 8. Email the Report

In [46]:
import smtplib
from email.message import EmailMessage

def send_portfolio_report_email(
  smtp_user,
  smtp_password,
  smtp_server,
  smtp_port,
  receiver_email,
  report_path
):

  # Create email
  SENDER_NAME = "Automated Portfolio Analytics System"
  msg = EmailMessage()
  msg["From"] = f"{SENDER_NAME} <{smtp_user}>"
  msg["To"] = receiver_email
  msg["Subject"] = f"Portfolio Report: {datetime.now().strftime('%Y-%m-%d')} (UTC)"
  msg.set_content(
"""
Hello,

Please find attached the latest automated portfolio performance & risk report.

This report includes:
• Portfolio-level performance and risk metrics
• Asset-level KPIs
• Price, return, drawdown, volatility, and correlation visualizations

This email was generated automatically.

Best regards,
Automation Team
"""
  )

  # Attach PDF report
  if report_path and os.path.exists(report_path):
      with open(report_path, "rb") as f:
          file_data = f.read()
          file_name = os.path.basename(report_path)

      msg.add_attachment(
          file_data,
          maintype="application",
          subtype="pdf",
          filename=file_name
      )
  else:
      raise FileNotFoundError("PDF report not found. Email not sent.")

  # Send email via SMTP
  with smtplib.SMTP(smtp_server, smtp_port) as server:
      server.starttls()
      server.login(smtp_user, smtp_password)
      server.send_message(msg)

  print("Email sent successfully!")

# Call the function to send the email
send_portfolio_report_email(
    smtp_user=EMAIL_USER,
    smtp_password=EMAIL_PASS,
    smtp_server=SMTP_SERVER,
    smtp_port=SMTP_PORT,
    receiver_email=RECEIVER_EMAIL,
    report_path=report_path
)

Email sent successfully!


Once the PDF report is generated, the next step is distributing it automatically via email. This ensures stakeholders or team members receive the latest portfolio insights in real time, without manual effort.

Key Features of this Step:
-  Custom Sender Name: The email appears from `"Automated Portfolio Analytics System"` instead of just the email address, giving a professional touch.
-  Dynamic Subject Line: Includes the current date in the subject for version tracking.
-  Pre-formatted Email Body: Highlights what the report includes such as portfolio metrics, asset KPIs, and visualizations.
-  PDF Attachment: Automatically attaches the generated PDF report to the email.
-  SMTP Authentication: Securely logs in and sends the email via a specified SMTP server (e.g., Gmail SMTP with TLS).
-  Error Handling: Checks if the PDF exists before sending to prevent sending empty emails.

Outcome: Stakeholders receive a fully formatted, up-to-date portfolio report directly in their inbox, enabling continuous monitoring and faster decision-making.

